In [ ]:
import requests
from matplotlib import pyplot as plt

In [ ]:
with open("../apikey.txt", "r") as IN:
    APIKEY = IN.read()

In [ ]:
APIKEY

## Wie viele Objekte je Sektor gibt es?

### Liste der Sektoren

In [ ]:
headers = {
    'Accept' : 'application/json',
    'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

res = requests.get('https://api.deutsche-digitale-bibliothek.de/institutions/sectors', headers = headers)

sectorList = [_['value'] for _ in res.json()]

### Zahl der Items je Sektor

In [ ]:
def itemsPerSector(sector):
    res = requests.get("https://api.deutsche-digitale-bibliothek.de/search",
             params = {
                 'query' : '*',
                 'facet' : 'sector_fct',
                 'sector_fct' : sector
             },
            headers = headers
                      )
    return res.json()

In [ ]:
sectorStatistics = { s : int(itemsPerSector(s)['numberOfResults']) for s in sectorList }

In [ ]:
xs,ys = zip(*sectorStatistics.items())
plt.bar(xs, ys)
plt.title("Zahl der Objekte je Sektor")
plt.show()

## Datenabzug nach Suchkriterien

Wie ziehe ich mir alle Datensätze zu Käthe Kollwitz?

In [ ]:
endpunkt = "https://api.deutsche-digitale-bibliothek.de/search"

headers = {
'Accept' : 'application/json',
'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

parameters = {
    'facet' : 'affiliate_fct_role',
    'affiliate_fct_role' : 'Käthe Kollwitz'
}

res = requests.get(endpunkt, headers = headers, params = parameters)

In [ ]:
for doc in res.json()['results'][0]['docs']:
    item = requests.get(f'https://api.deutsche-digitale-bibliothek.de/items/{doc["id"]}', headers = headers).json()
    print(item)
    break # höre nach dem ersten Durchlauf auf

## Iteration durch Ergebnismenge ('Paging')

### Ineffiziente Methode (mit Hochzählen von `offset`)

In [ ]:
endpoint = "https://api.deutsche-digitale-bibliothek.de/search"
params = {
  "query": "dante alighieri",
  "type_fct": "mediatype_002",
  "facet": [
    "type_fct"
  ]
}

headers = {
  "Accept": "application/json",
  "Authorization": f'OAuth oauth_consumer_key="{APIKEY}"'
}

response = requests.get(endpoint, headers=headers, params=params)

Wie viele Objekte umfasst die Ergebnismenge?

In [ ]:
numberOfResults = response.json().get('numberOfResults')
print(numberOfResults)

Wir wollen jetzt in Zehnerschritten durch die Anfrage iterieren und nach und nach alle IDs in Liste `collectedIDs` einsammeln.

In [ ]:
collectedIDs = list()

# in Funktion gekapselter API-Call

def callDDBAPI(offset):
    endpoint = "https://api.deutsche-digitale-bibliothek.de/search"
    params = {
      "query": "dante alighieri",
      "type_fct": "mediatype_002",
      "facet": [
        "type_fct"
      ],
    "sort" : "ALPHA_ASC",
    "offset" : offset, # wird bei jeder Iteration erhöht
    "rows" : 10 # bleibt stabil, es sollen jedesmal 10 Datensätze abgerufen werden
    }

    headers = {
      "Accept": "application/json",
      "Authorization": f'OAuth oauth_consumer_key="{APIKEY}"'
    }
    
    response = requests.get(endpoint, headers = headers, params = params)
    IDs = [_.get('id') for _ in response.json().get('results')[0].get('docs')]
    return IDs   

In [ ]:
offset = 0
while offset <= numberOfResults:
    collectedIDs.extend(callDDBAPI(offset))
    offset += 10

In [ ]:
print(collectedIDs)

### Alternative Methode (mit `cursorMark`)

In [ ]:
def callDDBAPI(cursorMark):
    endpoint = "https://api.deutsche-digitale-bibliothek.de/search"
    params = {
      "query": "dante alighieri",
      "type_fct": "mediatype_002",
      "facet": [
        "type_fct"
      ],
    "sort" : "ALPHA_ASC",
    "cursorMark" : cursorMark, # wird bei jeder Iteration erhöht
    "rows" : 100 # bleibt stabil, es sollen jedesmal 10 Datensätze abgerufen werden
    }
    
    headers = {
      "Accept": "application/json",
      "Authorization": f'OAuth oauth_consumer_key="{APIKEY}"'
    }
    
    response = requests.get(endpoint, headers = headers, params = params)
    #IDs = [_.get('id') for _ in response.json().get('results')[0].get('docs')]
    return response   
    

In [ ]:
collectedIDs_cursorMark = list()

res = callDDBAPI('*')
IDs = [_.get('id') for _ in res.json().get('results')[0].get('docs')]
collectedIDs_cursorMark.extend(IDs)

firstCursorMark = res.json().get('nextCursorMark').strip()
nextCursorMark = res.json().get('nextCursorMark').strip()

while True:
    res = callDDBAPI(nextCursorMark)
    IDs = [_.get('id') for _ in res.json().get('results')[0].get('docs')]
    collectedIDs_cursorMark.extend(IDs)
    if nextCursorMark == res.json().get('nextCursorMark').strip():
        print("Done!")
        break
    else:
        nextCursorMark = res.json().get('nextCursorMark').strip()

In [ ]:
len(collectedIDs_cursorMark)